In [1]:
using LinearAlgebra, Random, Statistics

In [2]:
# generate random data matrix
n,d = 6,4
X = randn(n,d)

# optional: give it linearly dependent columns
X[:,3] = X[:,2]

6-element Array{Float64,1}:
 -1.4390366577132239
  0.6803124976385257
  0.9705025676320586
 -0.5300617417578417
 -0.7581949869736492
 -0.24097886941645416

In [3]:
U,σ,V = svd(X)

SVD{Float64,Float64,Array{Float64,2}}
U factor:
6×4 Array{Float64,2}:
  0.683959   0.189761    0.0129733  -0.0586252
 -0.419634   0.0372201  -0.222073   -0.4107
 -0.230941  -0.343193    0.855425    0.0527212
  0.075779   0.494218    0.365122   -0.721
  0.482334  -0.0727834   0.23328     0.144835
 -0.253742   0.771541    0.176152    0.53319
singular values:
4-element Array{Float64,1}:
 3.2487301820959424
 2.462361260800171
 1.6184289460071115
 2.0294575218679035e-16
Vt factor:
4×4 Array{Float64,2}:
 -0.153279  -0.565937  -0.565937  -0.579599
  0.539706  -0.395363  -0.395363   0.629359
  0.827782   0.152979   0.152979  -0.51766
  0.0        0.707107  -0.707107   0.0

In [4]:
U'*U

4×4 Array{Float64,2}:
  1.0           2.82606e-16  -1.74577e-16   8.86526e-17
  2.82606e-16   1.0          -1.12185e-16   4.92094e-17
 -1.74577e-16  -1.12185e-16   1.0          -3.36874e-17
  8.86526e-17   4.92094e-17  -3.36874e-17   1.0

In [5]:
V'*V

4×4 Array{Float64,2}:
  1.0          1.21612e-16  -4.75716e-17   1.01573e-16
  1.21612e-16  1.0           2.6856e-16    2.50044e-16
 -4.75716e-17  2.6856e-16    1.0          -1.44001e-16
  1.01573e-16  2.50044e-16  -1.44001e-16   1.0

In [6]:
σ

4-element Array{Float64,1}:
 3.2487301820959424
 2.462361260800171
 1.6184289460071115
 2.0294575218679035e-16

In [7]:
# if we have a linearly dependent column, 
# decomposition is just as good if we ignore the 0 in sigma and reduce r by 1
norm(X - U[:,1:3]*diagm(σ[1:3])*(V[:,1:3])')

2.8720148088613458e-15

In [8]:
# form data from noisy linear model
w♮ = randn(d)
y = X*w♮ + .1*randn(n);

In [9]:
# solve least squares problem to estimate w
w = V*diagm(σ.^(-1))*U'*y
# w = V[:,1:3]*diagm(σ[1:3].^(-1))*(U[:,1:3])'*y

4-element Array{Float64,1}:
  0.07075677218695144
  3.573965263096396e14
 -3.5739652630964175e14
 -0.4408930974242218

In [10]:
# how good is our estimate of w?
norm(w - w♮)

5.054350146521247e14

In [11]:
# compute mean square error
mean((y - X*w).^2)

0.11106675172269874

In [12]:
# let's use the shorthand
w_backslash = X \ y
norm(w_backslash - w)

5.054350146521265e14